
<div dir="rtl" align="right"> 

     توجه: در این پروژه قرار است اطلاعات سایت Darooyab.ir استخراج شود و به دلیل اینکه این سایت درخواست های خارج از ایران را مسدود می‌کند، بنابراین این نوتبوک  را نمی‌توانید با گوگل کولب اجرا کنید و حتما باید روی کامپیوتر شخصی خود اجرا نمایید.
    
    
    
    
</div>

<div dir="rtl" align="right"> 
    
    سایت گفته شده، یک سایت مرجع برای اطلاعات دارویی است که شامل حدود ۱۶۰۰ قلم دارو است. برای هر دارو دو نوع دسته بندی دارویی ذکر شده، اسم انگلیسی آن گفته شده و سپس توضیح های مختلفی ارائه شده است. قابل به ذکر است که توضیح های داروها قالب یکسانی ندارند یعنی ممکن است دارویی توضیح خاصی نداشته باشد اما داروی دیگر ۵۰ خط توضیح داشته باشد.
    
    ------------------------------------------------------------------------------------------------------------------------
    
</div>

<div dir="rtl" align="right"> 
    
    در ابتدا کتابخانه های مورد نیاز را فراخوانی میکنیم. تعدادی از کتابخانه ها هم نیاز به نصب دارند که قبل از فراخوانی آن‌ها، نصب را انجام میدهیم.
    
</div>

In [ ]:
!pip install hazm --quiet
!pip install nltk --quiet
!pip install freqtools --quiet

In [7]:
from __future__ import unicode_literals
import json
import numpy as np
import pandas as pd
import nltk
import codecs
import tqdm
import hazm
import re
import string
import itertools
import freqtools
import requests
from bs4 import BeautifulSoup
import json

<div dir="rtl" align="right"> 
    
    برای استخراج داده از این سایت، از کتابخانه های Requests و BeautifulSoup استفاده میکنیم.
    کتابخانه Request برای درخواست های HTTP Get و کتابخانه BeautifulSoup برای تجزیه اطلاعات HTML به کار میروند.
    
</div>

<div dir="rtl" align="right"> 
    
    در ابتدا آدرس پایه را تعریف کرده و سپس تمام اطلاعات آن صفحه را استخراج می‌کنیم. در سایت دارویاب، صفحه ای فهرست‌طور برای تمام دسته‌بندی های داروها وجود دارد. لذا این صفحه را به عنوان آدرس پایه استخراج میکنیم.
    
    سپس با تجزیه این صفحه، تمام لینک های درون آن را استخراج میکنیم که در حقیقت هر لینک، صفحه مربوط به یک دسته‌بندی خاص از دارو ها می‌باشد.
    
    حال به دلیل اینکه در هر صفحه لینک های متفرقه دیگری نیز وجود دارند که ما به آن‌ها نیاز نداریم، لذا با مشخص کردن یک الگوی زبان منظم، لینک هایی که مربوط به دسته‌بندی هستند را شناسایی و استخراج می‌کنیم و به دلیل اینکه از هر دسته ممکن است چند لینک موجود باشد، لینک های تکراری را نیز حذف میکنیم(با استفاده از تابع set)
    
</div>

In [8]:
url = 'https://www.darooyab.ir/DrugGroups'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

links_pure = []
for a in soup.find_all('a', href=True):
    links_pure.append(a['href'])
    
pattern = re.compile("/DrugGroups/.*")
filtered_link = list(set(filter(pattern.match, links_pure)))

<div dir="rtl" align="right">
    
   تا اینجا ما لینک مربوط به هر دسته را داریم. لذا به تک تک لینک ها درخواست داده و صفحات آن‌ها را استخراج میکنیم و تمام اطلاعات را ذخیره می‌کنیم. حال تک تک صفحات را باز کرده و لینک های درون آن‌ها را استخراج می‌کنیم که در حقیقت لینک صفحات مربوط به دارو‌ها هستند. در اینجا نیز لینک های متفرقه را به وسیله زبان های منظم کنار گذاشته و لینک های تکراری دارو ها را نیز حذف می‌کنیم. 
    
</div>

In [9]:
prefix = 'https://www.darooyab.ir'

for i in range(len(filtered_link)):
    filtered_link[i] = prefix + filtered_link[i]
    
links_pure = []
for i in range(len(filtered_link)):
    url = filtered_link[i]
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    for a in soup.find_all('a', href=True):
        links_pure.append(a['href'])
        
pattern = re.compile("/G-.*")
filtered_link2 = list(set(filter(pattern.match, links_pure)))

<div dir="rtl" align="right"> 
    
    در نهایت تمام اطلاعات مربوط به هر دارو را از لینک مربوطه استخراج کرده و تمام صفحات استخراج شده داروها را در یک متغیر ذخیره می‌کنیم.
    
</div>

In [10]:
prefix = 'https://www.darooyab.ir'

for i in range(len(filtered_link2)):
    filtered_link2[i] = prefix + filtered_link2[i]
    
final_pages = []
for i in range(len(filtered_link2)):
    url = filtered_link2[i]
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    final_pages.append(soup)

<div dir="rtl" align="right"> 
    
    در این بین ۳ صفحه متفرقه نیز ذخیره می‌شوند که آن هارا شناسایی کرده و به صورت دستی حذف می‌کنیم. مشاهده می‌شود که اطلاعات مربوط به ۱۶۱۷ قلم دارو استخراج شده است.
    
</div>

In [11]:
del final_pages[857]
del final_pages[859]
del final_pages[901]
len(final_pages)

1617

<div dir="rtl" align="right"> 
    
    در این مرحله از صفحات ذخیره شده، یک دیکشنری می‌سازیم که برای هر دارو اطلاعات استخراج شده آن را جدا کرده و مرتب کنیم. هر  entry در این دیکشنری، ۵ دسته   اطلاعات را در خود ذخیره می‌کند که به آن‌ها میپردازیم: 
    
</div>

<div dir="rtl" align="right"> 
    
    اطلاعات با کلید Persian name همان نام فارسی داروی مورد نظر است.
    
    اطلاعات با کلید English name همان نام انگلیسی داروی مورد نظر است.
    
    اطلاعات با کلید Martindale classification همان عنوان مربوط به دسته‌بندی مارتیندل داروها است.
    
    اطلاعات با کلید Treatment classification همان عنوان مربوط به دسته‌بندی درمانی داروها است.
    
    اطلاعات با کلید Special information اطلاعات تخصصی مربوط به هر دارو است.
    
</div>

In [12]:
myList = []
for daroo in range(len(final_pages)):
    if len(final_pages[daroo].find_all('div' , class_ ="JustifyClass")) == 0:
        continue
    
    if ('+' in str(final_pages[daroo].find_all('h1', class_ = 'EnglishNumericFont')[0].get_text())):
        continue
        
    if ('+' in str(final_pages[daroo].find_all('label', class_= 'EnglishNumericFont')[0].get_text())):
        continue
        
    if str(final_pages[daroo].find_all('h1', class_ = 'EnglishNumericFont')[0].get_text()) == '':
        continue
        
    myDict = {}
    temp = ''
    for i in range(len(final_pages[daroo].find_all('div' , class_ ="JustifyClass"))):
        temp = temp + '.' + final_pages[daroo].find_all('div' , class_ ="JustifyClass")[i].get_text()
        
    myDict = {
        'Persian Name': str(final_pages[daroo].find_all('h1', class_ = 'EnglishNumericFont')[0].get_text()),
        'English Name': str(final_pages[daroo].find_all('label', class_= 'EnglishNumericFont')[0].get_text()),
        'Martindale Classification': str(final_pages[daroo].find_all('a', class_ = 'ahref_Generic')[0].get_text()),
        'Treatment Classification': str(final_pages[daroo].find_all('label', class_= 'EnglishNumericFont')[1].get_text()),
        'Special Information': temp
    }
    myList.append(myDict)

<div dir="rtl" align="right"> 
    
     حال داده‌های پردازش شده را به شکل دیکشنری در یک فایل json به نام dataset ذخیره میکنیم که حجم آن 6.31 مگابایت می‌شود.
    
</div>

In [13]:
with open('./dataset.json', 'w', encoding='utf8') as fout:
    json.dump(myList , fout, ensure_ascii=False, indent=4)

<div dir="rtl" align="right"> 
    
    حال برای کاوش در متن این اطلاعات، فایل ذخیره شده را باز کرده و تبدیل به دیتافریم می‌کنیم. 
    
</div>

In [14]:
with open('./dataset.json', 'rb') as json_file:
    data = json.load(json_file , encoding="utf8")

In [15]:
data_dataframe = pd.DataFrame(data)

<div dir="rtl" align="right"> 
    
    در ابتدا به عنوان پیش‌پردازش اولیه داده‌ها(ما فقط داده های با کلید Special information را پردازش می‌کنیم زیرا مابقی فیلدها تمیز و درست هستند و نیازی به پیش پردازش ندارند)
    
    تابعی تعریف کرده که به وسیله زبان های منظم، تمام حروف انگلیسی و فاصله های اضافی و نقطه‌گذاری هارا حذف کند.
    
</div>

In [16]:
def preprocess1(text):
    text = re.sub("[a-zA-Z]","",text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\t\n\d]+','',text)
    return text

<div dir="rtl" align="right"> 
    
    تابع را روی داده ها اعمال کرده و سپس خروجی آن‌ها را به وسیله تابع نرمالایزر کتابخانه هضم، استاندارد‌سازی می‌کنیم.
    
</div>

In [17]:
normalizer = hazm.Normalizer()
for i in range(len(data_dataframe['Special Information'])):
    data_dataframe['Special Information'][i] = preprocess1(data_dataframe['Special Information'][i])
    data_dataframe['Special Information'][i] = normalizer.normalize(data_dataframe['Special Information'][i])

<div dir="rtl" align="right"> 
    
    سپس تمام داده‌ها را به وسیله تابع توکنایزر کتابخانه هضم، توکنایز می‌کنیم.
    
</div>

In [18]:
data_dataframe_tokenize = data_dataframe
for i in range(len(data_dataframe_tokenize['Special Information'])):
    data_dataframe_tokenize['Special Information'][i] = hazm.word_tokenize(data_dataframe_tokenize['Special Information'][i])

<div dir="rtl" align="right"> 
    
    حال لیستی از تمام stopwords ها فارسی را بارگذاری کرده و به وسیله آن‌ها، تمام stopwords هارا از بین توکن‌ها حذف می‌کنیم.
    
</div>

In [19]:
with open("stopwords.txt", "r" ,encoding="utf8" ) as tf:
    stopwords = tf.read().split()

In [20]:
def preprocess2(text):
    text = [word for word in text if word not in stopwords]
    return text

In [21]:
data_dataframe_tokenize['Special Information'] = [preprocess2(y) for y in data_dataframe_tokenize['Special Information']]

<div dir="rtl" align="right"> 
    
    سپس به وسیله تابع Lemmatizer کتابخانه هضم، تمام توکن‌هارا ریشه‌یابی می‌کنیم.
    
</div>

In [22]:
lemmatizer = hazm.Lemmatizer()

In [23]:
data_dataframe_tokenize['Special Information'] = [[lemmatizer.lemmatize(y) for y in x] for x in data_dataframe_tokenize['Special Information']]

<div dir="rtl" align="right">
    
    سپس تمام توکن های مربوط به تمام داروها را در یک متغیر جمع کرده تا روی آن‌ها اعمال آماری را انجام دهیم.
    
</div>

In [24]:
all_tokens = list(itertools.chain(*data_dataframe_tokenize['Special Information']))

<div dir="rtl" align="right"> 
    
    در این مرحله، ۵۰ توکن پرتکرار از بین تمامی توکن هارا استخراج کرده و نمایش می‌دهیم.
    
</div>

In [26]:
most_frequent = nltk.FreqDist(eval(F"all_tokens")).most_common(50)

In [34]:
most_frequent

[('دارو', 10162),
 ('مصرف', 6222),
 ('افزایش', 5120),
 ('کاهش', 4185),
 ('اثرات', 3737),
 ('صورت', 3409),
 ('درمان', 3217),
 ('بیمار', 3165),
 ('توسط', 2843),
 ('خون', 2615),
 ('درد', 2304),
 ('تداخل', 1996),
 ('ضد', 1809),
 ('قلب', 1771),
 ('کبد', 1655),
 ('ساعت', 1640),
 ('گیاه', 1534),
 ('شد#شو', 1475),
 ('سیستمیک', 1440),
 ('ادرار', 1343),
 ('مهارکننده', 1291),
 ('پوست', 1279),
 ('عروق', 1261),
 ('عفونت', 1193),
 ('تزریق', 1175),
 ('عضلانی', 1163),
 ('استفاده', 1160),
 ('موضع', 1149),
 ('اسید', 1078),
 ('تشدید', 1075),
 ('مرکزی', 1068),
 ('اختلال', 1057),
 ('جذب', 1052),
 ('سطح', 1044),
 ('حساسیت', 1039),
 ('مهار', 1020),
 ('التهاب', 1013),
 ('تجویز', 996),
 ('اعصاب', 979),
 ('گوارش', 966),
 ('متابولیسم', 959),
 ('شایع', 951),
 ('فشار', 951),
 ('عوارض', 944),
 ('احتیاط', 937),
 ('شدید', 911),
 ('واکسن', 889),
 ('دوز', 879),
 ('سلول', 871),
 ('طولانی', 871)]

<div dir="rtl" align="right"> 
    
    سپس تعداد کل کلمات، تعداد کل کلمات یکتا، میانگین طول کلمات و بلندترین کلمه را استخراج کرده و نمایش می‌دهیم.
    
</div>

In [28]:
print ('%-16s' % 'Number of words', '%-16s' % len(all_tokens))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_tokens)))
avg=np.sum([len(word) for word in all_tokens])/len(all_tokens)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_tokens[np.argmax([len(word) for word in all_tokens])])

Number of words  364216          
Number of unique words 20721           
Average word length 5.839575965910339
Longest word     نارسیسینایزوکوئرستینایزورامنتینکوئرستینروتینوزیدایزورامنتین


<div dir="rtl" align="right"> 
    
    سپس ۲۰ کلمه پرتکرار در اطلاعات تخصصی هر دارو را استخراج کرده و به عنوان یک ستون جدید برای هر دارو ثبت می‌کنیم. 
    
    با اینکار ما می‌توانیم فقط با نگاه کردن به کلمات پرتکرار هر دارو، از کاربرد و دسته‌بندی و عملکرد هر دارو تا حدودی مطلع شویم.
    
</div>

In [29]:
data_dataframe_tokenize['Most frequent tokens'] = [nltk.FreqDist(eval(F"y")).most_common(20) for y in data_dataframe_tokenize['Special Information']]

<div dir="rtl" align="right"> 
    
    با تغییر اولین ورودی تابع iloc میتوانید کلمات پرتکرار هر دارو را مشاهده کنید.
    
</div>

In [30]:
data_dataframe_tokenize.iloc[100,5]

[('دارو', 17),
 ('مصرف', 16),
 ('افزایش', 7),
 ('ویتامین', 7),
 ('کاهش', 6),
 ('پلاکت', 6),
 ('صورت', 5),
 ('تیکلوپیدین', 4),
 ('گوارش', 4),
 ('جذب', 4),
 ('غذا', 4),
 ('میلی', 4),
 ('گرم', 4),
 ('روز', 4),
 ('خون', 4),
 ('اثرات', 4),
 ('توسط', 4),
 ('ضد', 4),
 ('ای', 4),
 ('خونریزی', 3)]

<div dir="rtl" align="right"> 
    
    در این قسمت سیاق را می‌بینیم.
    
</div>

In [31]:
text = nltk.Text(all_tokens)
text.concordance('پوست')

Displaying 25 of 25 matches:
جویز دارو عوارض شیوع عبارتند کبود رنگ پوست اختلال بینایی ویا دوبینی تشنج سرگیجه 
رو توسط ریتوکسیمببثژ داخل مثانهای تست پوست کوکسیدیوئید ایمیت نیوولومب پیدوتیمود 
ی خواب رفتگی اندام گذراپوستی خارش راش پوست کهیرگوارشی اسهال تهوع استفراغمشخصات ت
ب مرکزی سردرد خوابید#خواب خواب آلودگی پوست افزایش تعریق گوارش خشکی پوست تهوع کاه
ب آلودگی پوست افزایش تعریق گوارش خشکی پوست تهوع کاهش اشتها درد شکم استفراغ یبوست
ات ناپایدار آژیتاسیون قرار احساس سرما پوست کنده پوست راش پوست خارش کهیر غدد ریز 
ر آژیتاسیون قرار احساس سرما پوست کنده پوست راش پوست خارش کهیر غدد ریز متابولیسمک
ون قرار احساس سرما پوست کنده پوست راش پوست خارش کهیر غدد ریز متابولیسمکاهش وزن ک
ط پیمترکسید دی سدیمبثژ داخل مثانه تست پوست کوکسیدیوئید ایمیت لنوگراستیم لیپگفیلگ
وره پا بیمار موکوزیت اسهال سم ریوی سم پوست پا کورتیکواستروئیدها کاهش واکنش جلد ت
مچنین عصاره ریشه ختم درمان سوختگی زخم پوست استاز گیاه سوزش معده سوء هاضمه زخم مع
اغ کاهش اشتها درد شکم یبوست سوء هاضمه پوست راش خارش خون لوکوپنی عصبی عضلانی اسکل

In [32]:
text[0]

'دارو'

<div dir="rtl" align="right"> 
    
    این هم یک شمای کلی از داده‌های ذخیره شده و پردازش ‌های انجام شده بر روی آن‌ها. داده های اصلی را هم می‌توانید در فایل json ذخیره شده مشاهده کنید.
    
</div>

In [33]:
data_dataframe_tokenize

,Persian Name,English Name,Martindale Classification,Treatment Classification,Special Information,Most frequent tokens
0,سیپروفلوکساسین چشمی,Ciprofloxacin-Ophthalmic,داروهای ضد باکتری,فراورده های چشمی-گوشی-بینی-حلق,"[دارو, درمان, عفونت, سطح, چشم, بویژه, زخم, قرن...","[(چشم, 7), (دارو, 4), (قرنیه, 2), (مصرف, 2), (..."
1,امیکاسین,Amikacin,داروهای ضد باکتری,داروهای ضد عفونت,"[دارو, درمان, عفونت, باکتری, گرم, منفی, مقاوم,...","[(دارو, 15), (سم, 7), (مصرف, 6), (بیمار, 6), (..."
2,تتراکائین,Tetracaine,بیحس کننده های موضعی,فراورده های چشمی-گوشی-بینی-حلق,"[تتراکائین, ایجاد, حس, موضع, منطق, عنکبوتیه, م...","[(دارو, 14), (مصرف, 9), (موضع, 7), (صورت, 6), ..."
3,ویتامین ب2,Vitamin B2,عوامل تغذیه ای و ویتامین ها(مکمل ها),ویتامین ها,"[موارد, مصرفپیشگیری, کمبود, ریبو, فلاوین, درما...","[(کمبود, 2), (ریبو, 2), (فلاوین, 2), (موارد, 1..."
4,برنجاسف,Achillea wilhelmsii C.Koch,داروهای گیاهی,فاقد طبقه بندی فارماکولوژی-درمانی AHFS,"[نام, علم, گیاه, خانواده, گیاه, استفاده, اندام...","[(گیاه, 2), (نام, 1), (علم, 1), (خانواده, 1), ..."
...,...,...,...,...,...,...
1116,نالتروکسون,Naltrexone,آنتی دوت ها،شلات کننده ها و آنتاگونیست ها,داروهای موثر بر سیستم اعصاب مرکزی,"[دارو, عنوان, عامل, کمک, درمان, معتادانی, کار,...","[(افزایش, 11), (دارو, 10), (کاهش, 9), (درد, 9)..."
1117,اتیدرونیک اسید,Etidronic Acid,داروهای استخوان ساز,عوامل درمانی متفرقه,"[دارو, صورت, خوراک, درمان, بیمار, پاژه, پیشگیر...","[(دارو, 13), (استخوان, 12), (کاهش, 8), (کلسیم,..."
1118,ماپروتیلین,Maprotiline,داروهای ضد افسردگی,داروهای موثر بر سیستم اعصاب مرکزی,"[دارو, بهبود, بیمار, افسردگی, تسکین, تجویز, می...","[(دارو, 20), (مصرف, 16), (اثرات, 7), (نوبت, 6)..."
1119,سیمواستاتین,Simvastatin,داروهای قلبی عروقی,داروهای قلبی-عروقی,"[سیمواستاتین, عنوان, دارو, کمک, همراه, رژیم, غ...","[(دارو, 16), (مصرف, 14), (افزایش, 9), (درمان, ..."
